In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("cat_dog1.csv")
df.head()

,Unnamed: 0,image,labels
0,0,dog.624.jpg,1
1,1,cat.170.jpg,0
2,2,dog.419.jpg,1
3,3,dog.19.jpg,1
4,4,cat.227.jpg,0


In [ ]:

X = df[['image']]
y = df['labels']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Install transformers library
!pip install transformers

# Clone the CLIP repository from GitHub
!git clone https://github.com/openai/CLIP

# Change directory to the CLIP repository
%cd CLIP

# Install the CLIP package
!pip install .

# Install additional dependencies (such as torch, torchvision, and ftfy)
!pip install torch torchvision ftfy


fatal: destination path 'CLIP' already exists and is not an empty directory.
/content/CLIP
Processing /content/CLIP
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=edc1507e8eec3c1936df33c6d31bbd3b4b1b84afd887434e29a53771418a1565
  Stored in directory: /tmp/pip-ephem-wheel-cache-1h_skj5s/wheels/04/71/64/2dc165f309b37a2995d8a283d3587d1ed48a02dcd7f56322d9
Successfully built clip
  Attempting uninstall: clip
    Found existing installation: clip 1.0
    Uninstalling clip-1.0:
      Successfully uninstalled clip-1.0


In [ ]:
from google.colab import files
import zipfile
import io

# Upload a ZIP file containing your images
uploaded = files.upload()

# Extract the uploaded ZIP file
zip_file_path = next(iter(uploaded))
with zipfile.ZipFile(io.BytesIO(uploaded[zip_file_path]), 'r') as zip_ref:
    zip_ref.extractall('/content/dataset_folder')  # Replace with your desired folder path

# Update the image paths
dataset_folder_path = '/dataset_folder'
train_image_paths = [dataset_folder_path + '/' + image_path for image_path in X_train['image']]
test_image_paths = [dataset_folder_path + '/' + image_path for image_path in X_test['image']]


In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import clip
from io import BytesIO
from tqdm import tqdm
import os

In [ ]:

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)

# Set the model to evaluation mode
#model.eval()

In [ ]:
def get_image_embedding(image_path):
    image = transform(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
    return image_features.cpu().numpy()
    # Update the image paths
dataset_folder_path = '/content/dataset_folder/cat_dog1'
train_image_paths = [dataset_folder_path + '/' + image_path for image_path in X_train['image']]
test_image_paths = [dataset_folder_path + '/' + image_path for image_path in X_test['image']]

train_image_embeddings = {}
for image_path in tqdm(train_image_paths, desc="Processing Train Images"):
    try:
        train_image_embeddings[image_path] = get_image_embedding(image_path)
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")

test_image_embeddings = {}
for image_path in tqdm(test_image_paths, desc="Processing Test Images"):
    try:
        test_image_embeddings[image_path] = get_image_embedding(image_path)
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")


In [ ]:
trainimage_paths = list(train_image_embeddings.keys())
trainembeddings = list(train_image_embeddings.values())

testimage_paths = list(test_image_embeddings.keys())
testembeddings = list(test_image_embeddings.values())

In [ ]:


trainembeddings_matrix = np.vstack(trainembeddings)
testembeddings_matrix = np.vstack(testembeddings)

In [ ]:

columns = [f"feature_{i}" for i in range(trainembeddings_matrix.shape[1])]
trainembeddings_df = pd.DataFrame(trainembeddings_matrix, index=trainimage_paths, columns=columns)

columns = [f"feature_{i}" for i in range(testembeddings_matrix.shape[1])]
testembeddings_df = pd.DataFrame(testembeddings_matrix, index=testimage_paths, columns=columns)

In [ ]:
trainembeddings_df

In [ ]:
testembeddings_df

In [ ]:
y_train = df.loc[X_train.index, 'labels'].values
y_test = df.loc[X_test.index, 'labels'].values

In [ ]:
print(f"Length of train_image_embeddings: {len(train_image_embeddings)}")
print(f"Length of X_train: {len(X_train)}")
print(f"Length of y_train: {len(y_train)}")


class_0_mean = trainembeddings_df[y_train == 0].mean()
class_1_mean = trainembeddings_df[y_train == 1].mean()
class_0_cov = trainembeddings_df[y_train == 0].cov()
class_1_cov = trainembeddings_df[y_train == 1].cov()
C_inv = np.linalg.inv(class_0_cov + class_1_cov)


In [ ]:
# Project test set embeddings using Fisher's Linear Discriminant
test_embeddings_projected = np.dot(testembeddings_matrix, w)

# Assuming a threshold for binary classification (adjust as needed)
threshold = 0.5
predictions = (test_embeddings_projected > threshold).astype(int)

# Evaluate the predictions
accuracy = np.mean(predictions == y_test)
print(f"Accuracy: {accuracy}")
